In [1]:
import os
from model_lib import *
from load_data_lib import *
from train_analyze_lib import *

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

Device: cuda
cuda


In [3]:
def main():
    """Main function with flexible path handling"""
    
    print("Plant Disease Detection - Multi-Model Error Analysis")
    print("=" * 60)
    
    # FLEXIBLE PATH CONFIGURATION
    current_dir = os.getcwd()
    print(f"Current working directory: {current_dir}")
    
    # You can modify these paths:
    data_path = os.path.join(current_dir,"plant_data_tvt.pkl")
    models_dir = os.path.join(current_dir, "models")
    results_dir = os.path.join(current_dir, "results")

    print(f"Data path: {data_path}")
    print(f"Models directory: {models_dir}")
    print(f"Results directory: {results_dir}")
    
    # Create directories
    os.makedirs(models_dir, exist_ok=True)
    os.makedirs(results_dir, exist_ok=True)
    
    # Configuration parameters
    DATA_SUBSET_RATIO = 1  # Use 100% of data for quick testing
    EPOCHS = 40  # Number of epochs per model
    
    print(f"Data subset ratio: {DATA_SUBSET_RATIO*100:.1f}%")
    print(f"Training epochs per model: {EPOCHS}")
    
    # Load data
    print("\nLoading data...")

    train_loader, val_loader, classes = load_data(data_path, subset_ratio=DATA_SUBSET_RATIO)
    print(f"Data loaded successfully!")
    print(f"Classes: {len(classes)}")
    print(f"Train batches: {len(train_loader)}")
    print(f"Val batches: {len(val_loader)}")
    
    # Get models for testing
    print("\nInitializing models...")
    all_models = get_all_models(num_classes=len(classes))
    
    print(f"Total models to train: {len(all_models)}")
    
    # Initialize result storage
    all_results = {}
    
    # Train models
    for i, (name, model) in enumerate(all_models.items()):
        print(f"\n{'='*80}")
        print(f"TRAINING MODEL {i+1}/{len(all_models)}: {name}")
        print(f"{'='*80}")
        
        try:
            # Move model to device
            model = model.to(device)
            
            # Train model with confusion matrix export
            results, current_val_loader = train_and_analyze(
                model, name, train_loader, val_loader, classes, results_dir, epochs=EPOCHS
            )
            all_results[name] = results
            
            # Save model
            model_path = os.path.join(models_dir, f"{name.lower()}_final.pth")
            torch.save(model.state_dict(), model_path)
            print(f"Model saved to: {model_path}")
            
            # Clear GPU memory
            del model
            torch.cuda.empty_cache()
            
        except Exception as e:
            print(f"ERROR training {name}: {e}")
            print(f"Skipping {name} and continuing with next model...")
            continue
    
    # Save comprehensive results
    if all_results:
        results_filename = 'all_models_comprehensive_results.json'
        results_path = os.path.join(results_dir, results_filename)
        
        with open(results_path, 'w') as f:
            json.dump(all_results, f, indent=2)
        
        print(f"\nComprehensive results saved: {results_path}")
        
        # Print final summary
        print(f"\n{'='*80}")
        print("FINAL SUMMARY")
        print(f"{'='*80}")
        
        for model_name, results in all_results.items():
            print(f"{model_name}:")
            print(f"  - Final Accuracy: {results['final_accuracy']:.2f}%")
            print(f"  - Confusion Matrix CSV: {results['confusion_matrix_csv']}")
            print(f"  - Classification Stats: {results['classification_stats_csv']}")
            print(f"  - Confusion Matrix Plot: {results['confusion_matrix_plot']}")
            print()
        
        # Best model
        best_model = max(all_results.items(), key=lambda x: x[1]['final_accuracy'])
        print(f"BEST MODEL: {best_model[0]} ({best_model[1]['final_accuracy']:.2f}%)")
        
    else:
        print("No models were successfully trained!")
    
    return all_results

if __name__ == "__main__":
    results = main()

Plant Disease Detection - Multi-Model Error Analysis
Current working directory: C:\Users\nhdkr\off_plant_di_err_analyze
Data path: C:\Users\nhdkr\off_plant_di_err_analyze\plant_data_tvt.pkl
Models directory: C:\Users\nhdkr\off_plant_di_err_analyze\models
Results directory: C:\Users\nhdkr\off_plant_di_err_analyze\results
Data subset ratio: 100.0%
Training epochs per model: 40

Loading data...
Data loaded successfully!
Classes: 38
Train batches: 1019
Val batches: 340

Initializing models...
Total models to train: 12

TRAINING MODEL 1/12: LeNet

Training LeNet...


Validation: 100%|████████████████████████████████████████████████████████████████████| 340/340 [00:19<00:00, 17.40it/s]


Epoch 1: Train 56.6%, Val 68.8%


Validation: 100%|████████████████████████████████████████████████████████████████████| 340/340 [00:20<00:00, 16.51it/s]


Epoch 2: Train 75.0%, Val 80.8%


...

Confusion matrix saved: C:\Users\nhdkr\off_plant_di_err_analyze\results\inceptionv3_confusion_matrix.csv
Classification statistics saved: C:\Users\nhdkr\off_plant_di_err_analyze\results\inceptionv3_classification_stats.csv


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [-1.7695861..1.9577874].
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [-2.117904..2.3760502].


Confusion matrix plot saved: C:\Users\nhdkr\off_plant_di_err_analyze\results\inceptionv3_confusion_matrix.png
Top-k Accuracy: {'top_1_acc': 99.59488076604364, 'top_3_acc': 99.97237823404843, 'top_5_acc': 99.98158548936561}


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [-1.8044444..1.3070468].
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [-1.7870152..2.3088455].
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [-1.7521569..1.923538].
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [-1.3687146..2.465708].
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [-1.8044444..1.7633053].
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [-1.9466565..2.3785625].
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [-1.68244..1

Saved 20 misclassified images to C:\Users\nhdkr\off_plant_di_err_analyze\results

InceptionV3 RESULTS:
----------------------------------------
Accuracy: 99.59%
Error rate: 0.41%
Files saved:
  - Confusion Matrix CSV: C:\Users\nhdkr\off_plant_di_err_analyze\results\inceptionv3_confusion_matrix.csv
  - Classification Stats: C:\Users\nhdkr\off_plant_di_err_analyze\results\inceptionv3_classification_stats.csv
  - Confusion Matrix Plot: C:\Users\nhdkr\off_plant_di_err_analyze\results\inceptionv3_confusion_matrix.png
Model saved to: C:\Users\nhdkr\off_plant_di_err_analyze\models\inceptionv3_final.pth

Comprehensive results saved: C:\Users\nhdkr\off_plant_di_err_analyze\results\all_models_comprehensive_results.json

FINAL SUMMARY
LeNet:
  - Final Accuracy: 97.25%
  - Confusion Matrix CSV: C:\Users\nhdkr\off_plant_di_err_analyze\results\lenet_confusion_matrix.csv
  - Classification Stats: C:\Users\nhdkr\off_plant_di_err_analyze\results\lenet_classification_stats.csv
  - Confusion Matrix Plot: